In [2]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import json
from time import sleep
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# 전체 페이지 수 가져오기
def get_page_num():

    url = 'https://www.bbcgoodfood.com/search/recipes?q=Vegan&sort=-popular'
    service = Service('../chromedriver/chromedriver.exe')

    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27'
    Options= webdriver.ChromeOptions()
    Options.add_argument('user-agent=' + user_agent)
    driver= webdriver.Chrome(service= service, options=Options)
    
    driver.get(url)
    sleep(3)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    page_nums = soup.find_all('a', {'class':'pagination-item'})
    nums = []
    for page_num in page_nums:
        nums.append(page_num.contents[-1])  
    num_list = list(range(1, int(nums[-1])+1))
    driver.close()

    return num_list

# 입력한 페이지의 전체 레시피 링크 가져오기
def get_links(i):
    link_list = list()
    #url = 'https://www.bbcgoodfood.com/search/recipes/page/' + str(i) + '/?q=Vegan&sort=-date'
    url = 'https://www.bbcgoodfood.com/search/recipes/page/' + str(i) + '/?q=Vegan&sort=-popular'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    lis = soup.find_all('div', {'class': 'standard-card-new__thumbnail'})

    for li in lis:
        link_list.append('https://www.bbcgoodfood.com' + li.contents[1].get('href'))

    return link_list

# 입력한 링크의 출처(site), 레시피명(title), 재료 리스트(ingredients), 조리시간(time), 분량(serving), 레시피(recipe),
# 영양 성분(nutrition), 댓글 리스트(comments), 이미지(image) 가져오기
def get_contents(url):
    global cookTime, calories, carbs, protein, total_fat
    contents = dict()
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    # 출처(site)
    # 함수에 입력받는 url 사용

    # 레시피명(title)
    title = soup.find('h1', {'class': 'heading-1'}).text

    # 재료 리스트(ingredients)
    ingredients = soup.find('section', {'class': 'recipe__ingredients col-12 mt-md col-lg-6'}).find_all('li')
    ing_list = list()
    for ingredient in ingredients:
        ing_list.append(ingredient.text)

    # 조리시간(time)
    cookTime_raw = soup.find_all('li', {'class': 'body-copy-small list-item'})
    if cookTime_raw:
        cookTime_list = list()
        for cookTime in cookTime_raw:
            cookTime_list.append(cookTime.text)
        for cookTime in cookTime_list:
            if 'Cook:' in cookTime:
                cookTime = cookTime.split('Cook:')[1]
            else:
                pass
    else:
        cookTime = ''

    # 분량(serving)
    servings = soup.find('ul', {'class': 'post-header__row post-header__planning list list--horizontal'}).text
    val = "Serves" not in servings
    if val:
        servings = '1'
    else:
        servings = servings.split('Serves')[1].split()[0]
    servings = str(servings)

    # 레시피(recipe)
    recipe_raw = soup.find('ul', {'class': 'grouped-list__list list'}).find_all('div', {'class': 'editor-content'})
    recipe_list = list()
    for i in range(len(recipe_raw)):
        recipe_list.append(str(i + 1) + ". " + recipe_raw[i].text.replace('\n', ''))

    # 영양 성분(nutrition)
    nutrition = soup.find('table', {'class': 'key-value-blocks hidden-print mt-xxs'})
    if nutrition == None:
        pass
    else:
        calories = nutrition.find('td', text='kcal')
        if calories:
            calories = calories.find_next_sibling().text + 'kcal' 
        else:
            calories = ''
        carbs = nutrition.find('td', text='carbs')
        if carbs:
            carbs = carbs.find_next_sibling().text
        else:
            carbs = ''
        protein = nutrition.find('td', text='protein')
        if protein:
            protein = protein.find_next_sibling().text
        else:
            protein = ''
        total_fat = nutrition.find('td', text='fat')
        if total_fat:
            total_fat = total_fat.find_next_sibling().text
        else:
            total_fat = ''

    # 댓글 리스트(comments)
    # vegan_review = soup.find_all('p', {'class': 'mt-reset'})
    # com_list = list()
    # for comments in vegan_review:
    #     comments_raw = comments.text, comments.get_attribute('href')
    #     comments_list = comments_raw[0]
    #     com_list.append(comments_list)

    # 이미지(image)
    image = soup.find('div', {'class': 'image chromatic-ignore post-header-image image--fluid image--scaled-up'}) \
        .find('img', {'class': 'image__img'}).get('src')

    contents['site'] = url
    contents['title'] = title
    contents['ingredients'] = ing_list
    contents['time'] = cookTime
    contents['serving'] = servings + ' servings'
    contents['recipe'] = recipe_list
    contents['calories'] = calories
    contents['carbs'] = carbs
    contents['protein'] = protein
    contents['total_fat'] = total_fat
    # contents['comments'] = com_list
    contents['image'] = image

    return contents

# 전체 페이지 레시피 댓글 가져오기
def get_all_page_comment(num_list):
    total = dict()
    title_comments = list()
    a = 0

    with ThreadPoolExecutor(max_workers=10) as executor:
        for num in num_list:
            links = get_links(num)
            for link in links:
                print(link)
                content = executor.submit(get_contents, link)
                title_comments.append(content.result())

                a += 1
                print(a)

    date = datetime.today().strftime("%Y%m%d")
    
    total['date'] = date
    total['bbcgoodfood'] = title_comments
    return total

# 메인에서 실행
if __name__ == '__main__':
    # json 저장
    num_list = get_page_num()
    total = get_all_page_comment(num_list)

    date = datetime.today().strftime("%Y%m%d")
        
    with open(f'D:\\fruit_hada\\crawling\\crawling_result\\{date}_bbcgoodfood_all.json', 'w', encoding='utf-8-sig') as file:
        json.dump(total, file, indent="\t")            


https://www.bbcgoodfood.com/recipes/satay-sweet-potato-curry
1
https://www.bbcgoodfood.com/recipes/spinach-sweet-potato-lentil-dhal
2
https://www.bbcgoodfood.com/recipes/carrot-coriander-soup
3
https://www.bbcgoodfood.com/recipes/falafel-burgers-0
4
https://www.bbcgoodfood.com/recipes/vegan-banana-bread
5
https://www.bbcgoodfood.com/recipes/tomato-soup
6
https://www.bbcgoodfood.com/recipes/kadala-curry
7
https://www.bbcgoodfood.com/recipes/one-pot-mushroom-potato-curry
8
https://www.bbcgoodfood.com/recipes/spicy-roasted-parsnip-soup
9
https://www.bbcgoodfood.com/recipes/roasted-aubergine-tomato-curry
10
https://www.bbcgoodfood.com/recipes/vegan-chilli
11
https://www.bbcgoodfood.com/recipes/vegan-lemon-cake
12
https://www.bbcgoodfood.com/recipes/kidney-bean-curry
13
https://www.bbcgoodfood.com/recipes/best-ever-chunky-guacamole
14
https://www.bbcgoodfood.com/recipes/focaccia
15
https://www.bbcgoodfood.com/recipes/chunky-oven-chips
16
https://www.bbcgoodfood.com/recipes/tomato-basil-sauc

135
https://www.bbcgoodfood.com/recipes/vegan-mug-cake
136
https://www.bbcgoodfood.com/recipes/roast-aubergine-coconut-curry
137
https://www.bbcgoodfood.com/recipes/slow-cooker-spiced-root-lentil-casserole
138
https://www.bbcgoodfood.com/recipes/singapore-noodles-tofu
139
https://www.bbcgoodfood.com/recipes/avocado-panzanella
140
https://www.bbcgoodfood.com/recipes/rhubarb-cordial
141
https://www.bbcgoodfood.com/recipes/baked-banana-porridge
142
https://www.bbcgoodfood.com/recipes/crispy-new-potato-bake
143
https://www.bbcgoodfood.com/recipes/falafel-burgers
144
https://www.bbcgoodfood.com/recipes/coconut-cashew-butternut-squash-curry
145
https://www.bbcgoodfood.com/recipes/vegan-flapjacks
146
https://www.bbcgoodfood.com/recipes/mustard-potato-salad
147
https://www.bbcgoodfood.com/recipes/tomato-pepper-bean-one-pot
148
https://www.bbcgoodfood.com/recipes/goan-style-vegetable-curry-kitchari
149
https://www.bbcgoodfood.com/recipes/poached-plums
150
https://www.bbcgoodfood.com/recipes/faj

264
https://www.bbcgoodfood.com/recipes/nuts-seeds-granola
265
https://www.bbcgoodfood.com/recipes/coconut-banana-pancakes
266
https://www.bbcgoodfood.com/recipes/tropical-breakfast-smoothie
267
https://www.bbcgoodfood.com/recipes/vegan-sausage-rolls
268
https://www.bbcgoodfood.com/recipes/vegan-breakfast-muffins
269
https://www.bbcgoodfood.com/recipes/vegan-christmas-pudding
270
https://www.bbcgoodfood.com/recipes/miso-aubergines
271
https://www.bbcgoodfood.com/recipes/yellow-lentil-coconut-curry-cauliflower
272
https://www.bbcgoodfood.com/recipes/lentil-sweet-potato-cottage-pie
273
https://www.bbcgoodfood.com/recipes/charred-cauliflower-lemon-caper-orzo
274
https://www.bbcgoodfood.com/recipes/cauliflower-chickpea-pilaf
275
https://www.bbcgoodfood.com/recipes/classic-guacamole
276
https://www.bbcgoodfood.com/recipes/sweet-sour-tofu
277
https://www.bbcgoodfood.com/recipes/mushroom-sweet-potato-wellington
278
https://www.bbcgoodfood.com/recipes/veggie-protein-chilli
279
https://www.bbcg

391
https://www.bbcgoodfood.com/recipes/smashed-roasties
392
https://www.bbcgoodfood.com/recipes/watermelon-smoothie
393
https://www.bbcgoodfood.com/recipes/spiced-roast-cauliflower-herby-rice
394
https://www.bbcgoodfood.com/recipes/squash-steaks-chestnut-cavolo-nero-pilaf
395
https://www.bbcgoodfood.com/recipes/sugar-syrup
396
https://www.bbcgoodfood.com/recipes/little-spicy-veggie-pies
397
https://www.bbcgoodfood.com/recipes/raw-lemon-cheesecake
398
https://www.bbcgoodfood.com/recipes/kentucky-fried-seitan
399
https://www.bbcgoodfood.com/recipes/salted-tomato-salad
400
https://www.bbcgoodfood.com/recipes/coronation-chickpea-sandwich-filler
401
https://www.bbcgoodfood.com/recipes/cheats-black-dhal
402
https://www.bbcgoodfood.com/recipes/brussels-sprouts-grain-bowl
403
https://www.bbcgoodfood.com/recipes/boxing-day-sprout-pad-thai
404
https://www.bbcgoodfood.com/recipes/fennel-roast-lemon-tomato-salad
405
https://www.bbcgoodfood.com/recipes/south-indian-aloo-masala-dosa-stuffing
406
ht

521
https://www.bbcgoodfood.com/recipes/halloween-toffee-apples
522
https://www.bbcgoodfood.com/recipes/chocolate-date-tart
523
https://www.bbcgoodfood.com/recipes/summer-carrot-tarragon-white-bean-soup
524
https://www.bbcgoodfood.com/recipes/bulgur-wheat-carrots-hazelnuts
525
https://www.bbcgoodfood.com/recipes/slow-cooker-breakfast-beans
526
https://www.bbcgoodfood.com/recipes/fruity-mocktail
527
https://www.bbcgoodfood.com/recipes/tangy-roasted-peppers
528
https://www.bbcgoodfood.com/recipes/vegan-meringues
529
https://www.bbcgoodfood.com/recipes/jewelled-squash-chestnut-mushroom-wreath
530
https://www.bbcgoodfood.com/recipes/gin-free-gt
531
https://www.bbcgoodfood.com/recipes/roasted-beets-plum-pecan-salad
532
https://www.bbcgoodfood.com/recipes/fruit-punch
533
https://www.bbcgoodfood.com/recipes/caramel-poached-peaches-blueberries
534
https://www.bbcgoodfood.com/recipes/vegan-fajitas
535
https://www.bbcgoodfood.com/recipes/low-sugar-granola
536
https://www.bbcgoodfood.com/recipes/

649
https://www.bbcgoodfood.com/recipes/zingy-rice-salad
650
https://www.bbcgoodfood.com/recipes/radish-cucumber-salad
651
https://www.bbcgoodfood.com/recipes/dosa
652
https://www.bbcgoodfood.com/recipes/cherries-rose-wine-vanilla-syrup
653
https://www.bbcgoodfood.com/recipes/miso-ramen
654
https://www.bbcgoodfood.com/recipes/baba-ganoush
655
https://www.bbcgoodfood.com/recipes/cosmopolitan-cocktail
656
https://www.bbcgoodfood.com/recipes/easy-chocolate-tart
657
https://www.bbcgoodfood.com/recipes/courgette-curry-with-lemon-rice
658
https://www.bbcgoodfood.com/recipes/stir-fried-sprouts-green-beans-lemon-pine-nuts
659
https://www.bbcgoodfood.com/recipes/superfood-pasta-salad
660
https://www.bbcgoodfood.com/recipes/rhubarb-chutney
661
https://www.bbcgoodfood.com/recipes/quince-jelly
662
https://www.bbcgoodfood.com/recipes/vegan-leek-potato-soup
663
https://www.bbcgoodfood.com/recipes/vegan-spinach-cherry-tomato-quiche
664
https://www.bbcgoodfood.com/recipes/herb-infused-roast-potatoes
6

776
https://www.bbcgoodfood.com/recipes/baked-falafel
777
https://www.bbcgoodfood.com/recipes/vegan-cupcakes
778
https://www.bbcgoodfood.com/recipes/carrot-and-orange-smoothie
779
https://www.bbcgoodfood.com/recipes/vegan-eggnog
780
https://www.bbcgoodfood.com/recipes/vegan-toad-in-the-hole-with-red-onion-balsamic-gravy
781
https://www.bbcgoodfood.com/recipes/homemade-croutons
782
https://www.bbcgoodfood.com/recipes/keto-smoothie
783
https://www.bbcgoodfood.com/recipes/roasted-asparagus
784
https://www.bbcgoodfood.com/recipes/fresh-orange-lollies
785
https://www.bbcgoodfood.com/recipes/caramelised-cranberry-relish
786
https://www.bbcgoodfood.com/recipes/cucumber-salad-pickled-red-onions
787
https://www.bbcgoodfood.com/recipes/seeded-flatbreads
788
https://www.bbcgoodfood.com/recipes/stewed-apple
789
https://www.bbcgoodfood.com/recipes/matcha-latte
790
https://www.bbcgoodfood.com/recipes/vegan-gingerbread
791
https://www.bbcgoodfood.com/recipes/mini-nut-roasts-candied-carrots
792
https:

906
https://www.bbcgoodfood.com/recipes/cherry-bakewell-cocktail
907
https://www.bbcgoodfood.com/recipes/frozen-blackberry-smoothie
908
https://www.bbcgoodfood.com/recipes/hummus-snack-packs
909
https://www.bbcgoodfood.com/recipes/rum-old-fashioned
910
https://www.bbcgoodfood.com/recipes/peach-cardamom-jam
911
https://www.bbcgoodfood.com/recipes/ful-medames
912
https://www.bbcgoodfood.com/recipes/peach-cardamom-jam
913
https://www.bbcgoodfood.com/recipes/easy-cranberry-orange-relish
914
https://www.bbcgoodfood.com/recipes/pearled-spelt-salad-peas-gooseberries
915
https://www.bbcgoodfood.com/recipes/sicilian-spicy-pumpkin
916
https://www.bbcgoodfood.com/recipes/shredded-charred-corn-salad
917
https://www.bbcgoodfood.com/recipes/grapefruit-agave-pistachio-salad
918
https://www.bbcgoodfood.com/recipes/beastly-blackberry-bay-lemonade-mocktail
919
https://www.bbcgoodfood.com/recipes/orange-mint-salad
920
https://www.bbcgoodfood.com/recipes/thyme-chestnut-roast-potatoes
921
https://www.bbcgo

1035
https://www.bbcgoodfood.com/recipes/how-to-cook-brussels-sprouts
1036
https://www.bbcgoodfood.com/recipes/breakfast-peppers-chickpeas-with-tofu
1037
https://www.bbcgoodfood.com/recipes/charred-onion-tomato-salad
1038
https://www.bbcgoodfood.com/recipes/homemade-brown-sauce
1039
https://www.bbcgoodfood.com/recipes/tomatillo-salsa
1040
https://www.bbcgoodfood.com/recipes/vermouth-soda
1041
https://www.bbcgoodfood.com/recipes/spinach-green-bean-salad
1042
https://www.bbcgoodfood.com/recipes/spinach-raisins-pine-nuts
1043
https://www.bbcgoodfood.com/recipes/harissa
1044
https://www.bbcgoodfood.com/recipes/healthy-pumpkin-soup
1045
https://www.bbcgoodfood.com/recipes/vegan-nuggets
1046
https://www.bbcgoodfood.com/recipes/red-onion-peanut-butter-chilli
1047
https://www.bbcgoodfood.com/recipes/satay-tofu-skewers-garlic-ginger-pak-choi
1048
https://www.bbcgoodfood.com/premium/cauliflower-katsu-curry
1049
https://www.bbcgoodfood.com/recipes/chickpea-courgette-pilaf
1050
https://www.bbcgood

1167
https://www.bbcgoodfood.com/recipes/spicy-red-lentil-chilli-with-guacamole-rice
1168
https://www.bbcgoodfood.com/premium/umami-roast-potatoes
1169
https://www.bbcgoodfood.com/recipes/chana-daal-chaat-with-tamarind-herbs
1170
https://www.bbcgoodfood.com/recipes/ajwain-puri
1171
https://www.bbcgoodfood.com/recipes/tapenade-skordalia
1172
https://www.bbcgoodfood.com/recipes/matzo-granola-with-coconut-cashew-cranberry
1173
https://www.bbcgoodfood.com/recipes/roast-cauliflower-squash
1174
https://www.bbcgoodfood.com/recipes/crispy-banana-fritters-pisang-goreng-wijen
1175
https://www.bbcgoodfood.com/recipes/dukkah-parsnips
1176
https://www.bbcgoodfood.com/recipes/oregano-roasties
1177
https://www.bbcgoodfood.com/recipes/dukkah-parsnips
1178
https://www.bbcgoodfood.com/recipes/spicy-bean-corn-dip
1179
https://www.bbcgoodfood.com/recipes/chicory-cups-with-avocado-pink-grapefruit-salsa
1180
https://www.bbcgoodfood.com/recipes/lemony-stoved-potatoes
1181
https://www.bbcgoodfood.com/recipes/

In [34]:
# 마지막 페이지 번호 찾기
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import json
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# # 전체 페이지 수 가져오기
# # def get_page_num():
# url = 'https://www.bbcgoodfood.com/search/recipes/page/2/?q=Vegan&sort=-popular'
# resp = requests.get(url)
# soup = BeautifulSoup(resp.text, 'html.parser')
# # sleep(5)
# print(soup)
# # page_nums = soup.find('div', {'class': 'pagination_item'})
# # max_num = page_nums.contents[-1]
# # print(max_num)

service = Service('../../chromedriver/chromedriver.exe')

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27'
Options= webdriver.ChromeOptions()
Options.add_argument('user-agent=' + user_agent)
driver= webdriver.Chrome(service= service, options=Options)


driver.get(url)
sleep(3)

soup = BeautifulSoup(driver.page_source, 'html.parser')
page_nums = soup.find_all('a', {'class':'pagination-item'})
nums = []
for page_num in page_nums:
    nums.append(page_num.contents[-1])

num_list = list(range(1, int(nums[-1])+1))
print(num_list)
driver.close()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]


In [53]:
# 없는 영양소는 패스, 있는 영양소만 가져오기
url = 'https://www.bbcgoodfood.com/recipes/gooseberry-chutney'
global cookTime, calories, carbs, protein, total_fat
contents = dict()
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

# 출처(site)
# 함수에 입력받는 url 사용

# 레시피명(title)
title = soup.find('h1', {'class': 'heading-1'}).text

# 재료 리스트(ingredients)
ingredients = soup.find('section', {'class': 'recipe__ingredients col-12 mt-md col-lg-6'}).find_all('li')
ing_list = list()
for ingredient in ingredients:
    ing_list.append(ingredient.text)

# 조리시간(time)
cookTime_raw = soup.find_all('li', {'class': 'body-copy-small list-item'})
if cookTime_raw:
    cookTime_list = list()
    for cookTime in cookTime_raw:
        cookTime_list.append(cookTime.text)
    for cookTime in cookTime_list:
        if 'Cook:' in cookTime:
            cookTime = cookTime.split('Cook:')[1]
        else:
            pass
else:
    cookTime = ''

# 분량(serving)
servings = soup.find('ul', {'class': 'post-header__row post-header__planning list list--horizontal'}).text
val = "Serves" not in servings
if val:
    servings = '1'
else:
    servings = servings.split('Serves')[1].split()[0]
servings = str(servings)

# 레시피(recipe)
recipe_raw = soup.find('ul', {'class': 'grouped-list__list list'}).find_all('div', {'class': 'editor-content'})
recipe_list = list()
for i in range(len(recipe_raw)):
    recipe_list.append(str(i + 1) + ". " + recipe_raw[i].text.replace('\n', ''))

# 영양 성분(nutrition)
nutrition = soup.find('table', {'class': 'key-value-blocks hidden-print mt-xxs'})
if nutrition == None:
    pass
else:
    calories = nutrition.find('td', text='kcal')
    if calories:
        calories = calories.find_next_sibling().text + 'kcal' 
    else:
        calories = ''
    carbs = nutrition.find('td', text='carbs')
    if carbs:
        carbs = carbs.find_next_sibling().text
    else:
        carbs = ''
    protein = nutrition.find('td', text='protein')
    if protein:
        protein = protein.find_next_sibling().text
    else:
        protein = ''
    total_fat = nutrition.find('td', text='fat')
    if total_fat:
        total_fat = total_fat.find_next_sibling().text
    else:
        total_fat = ''

# 댓글 리스트(comments)
# vegan_review = soup.find_all('p', {'class': 'mt-reset'})
# com_list = list()
# for comments in vegan_review:
#     comments_raw = comments.text, comments.get_attribute('href')
#     comments_list = comments_raw[0]
#     com_list.append(comments_list)

# 이미지(image)
image = soup.find('div', {'class': 'image chromatic-ignore post-header-image image--fluid image--scaled-up'}) \
    .find('img', {'class': 'image__img'}).get('src')

contents['site'] = url
contents['title'] = title
contents['ingredients'] = ing_list
contents['time'] = cookTime
contents['serving'] = servings + ' servings'
contents['recipe'] = recipe_list
contents['calories'] = calories
contents['carbs'] = carbs
contents['protein'] = protein
contents['total_fat'] = total_fat
# contents['comments'] = com_list
contents['image'] = image



{'site': 'https://www.bbcgoodfood.com/recipes/gooseberry-chutney', 'title': 'Gooseberry chutney', 'ingredients': ['2 red onions, thinly sliced ', '1 ½kg gooseberries', '2 fresh  bay leaves', '2 tsp mustard seeds', '1 heaped tbsp  coriander seeds, crushed ', '2 heaped tsp  salt', '650g soft brown sugar', '300ml cider vinegar'], 'time': '45 mins', 'serving': '1 servings', 'recipe': ['1. Put all the ingredients in a large, heavy-based saucepan, holding back 500g of the gooseberries. Cook over a medium heat until the sugar has dissolved, then bring the mixture to the simmer and cook, uncovered, for 30 mins. Add the remaining gooseberries and cook for a further 10-15 mins or until thickened.', '2. Spoon the chutney into five sterilised jars. Will keep for up to one month. Once opened, store in the fridge.'], 'calories': '40kcal', 'carbs': '0g', 'protein': '', 'total_fat': '0g', 'image': 'https://images.immediate.co.uk/production/volatile/sites/30/2020/08/gooseberry-chutney-d56cbfe.jpg?quali